In [ ]:
#@title Install libraries, imports and variables
#@markdown Please "Connect with Drive" before proceeding
#@markdown 
#@markdown Run only once

%tensorflow_version 1.x
!pip install anvil-uplink
!pip install -q gpt-2-simple
 
import tensorflow
import anvil.server
import gpt_2_simple as gpt2
from datetime import datetime
import numpy as np
import random
import tensorflow as tf
import tensorflow_hub as hub
import json
import requests
import os
import gc



if not os.path.isdir(f'./drive/'):
  print("Please Connect with Drive")
  from google.colab import drive
  drive.mount('/content/drive')

anvil_key = "" #@param {type:"string"}

generate_count = 0
anvil.server.connect(anvil_key)
 
print(datetime.now())
if not os.path.isdir('./checkpoint/run1'):
  print("Copy checkpoint...")
  gpt2.copy_checkpoint_from_gdrive(run_name='run1')
print(datetime.now())
 
@anvil.server.callable
def run(txt):
  #global sess
  #global generate_count
 
  txt = txt[:300]

  st =  datetime.now()

  print("Start time =", st)
 
  samples = 6
 
  tf.reset_default_graph()
  sess = gpt2.start_tf_sess(threads=-1)
  gpt2.load_gpt2(sess, run_name='run1')
 
  print('Generando...')
  arr = gpt2.generate(sess,
              length=300,
              temperature=0.7,
              prefix=f"<|startoftext|>[Informal]{txt}\n[Formal]",
              truncate = '<|endoftext|>',
              nsamples=samples,
              batch_size=samples,
              include_prefix=False,
              return_as_list=True)
  arr = list(set(arr))
 
  arr = [x.replace("[Formal]","").replace("[Informal]","").replace("<|startoftext|>","") for x in arr]
 
  lj = '\n'
  print(f"Input: {txt}\nOutputs:\n{lj.join(arr)}")
  sess.close()
  gc.collect()
 
  print("End time =", datetime.now())
  print("Duration =", datetime.now() - st)
  print("-"*50)
 
  return '\n\n'.join(arr)
 
def run2(txt):
  print("Running...")
  generated = generate(txt,samples=8)
  generated = [x.replace("[Formal]","").replace("[Informal]","").replace("<|startoftext|>","") for x in generated]
  return '\n\n'.join(generated)
 
@anvil.server.callable
def save(text,equiv):
  print("-"*50)
  print("Received text to save...")
  with open("/content/drive/MyDrive/dataset.txt","a+") as file:
    file.write(f"{text};{equiv}\n")
  print("-"*50)



#@markdown ____



In [ ]:
#@title Run to start server connection
anvil.server.wait_forever()
#@markdown ____